In [1]:
import json
from bs4 import BeautifulSoup

In [2]:
with open('projects_links.json') as file:
  linkinghost = json.load(file)

In [3]:
with open('data/project_data.json') as file:
  host = json.load(file)

In [4]:
hostList = []
chostList = []
linkinghostList = []

In [5]:
def splitPID(val):
    valhost = val.split("?")
    idhost = valhost[1].split("=")
    pureid = idhost[1]
    return pureid

In [6]:
def cleanSpecialChar(val):
    pureval = val.replace("\r", "").replace("\n", "").replace("\t", "")
    return pureval

In [7]:
def cleanSpecialCharAndSpace(val):
    pureval = val.replace("\r", "").replace("\n", "").replace("\t", "").replace(" ", "")
    return pureval

In [8]:
def write_data(passval, name):
    with open(name, 'w') as outfile:
        json.dump(passval, outfile, ensure_ascii=False, indent=4)

In [9]:
### Converting project_html to json objects
def getPHTML(val, idval):
    summary = {}    
# #     inserting Pid value to summary
    summary['Pid']=idval
    
# #     processing project summary information
    psumHeads = []
    psumVals = []
    psumdata = {}
    psum_soup = BeautifulSoup(val, 'html.parser')
    psumhostdata = psum_soup.find('div', attrs={'id': 'dsummary'}).find('table')
    psumheadvals = psumhostdata.findAll('th')
    psumvals = psumhostdata.findAll('td')
    for ps in psumheadvals:
        psumHeads.append(ps.text.replace(" ", ""))
    for count, psval in enumerate(psumvals):
        psumdata[psumHeads[count]]=cleanSpecialChar(psval.text)              
    psumVals.append(psumdata)
    summary['ProjectSummary']=psumVals
    
        
# #     processing budget part
    budgetHeads = []
    budgetVals = []
    budgetdata = {}
    soup = BeautifulSoup(val, 'html.parser')
    hostdata = soup.find('div', attrs={'id': 'p_budget'}).find('div').find('table')
    headvals = hostdata.findAll('th')
    vals = hostdata.findAll('td')
    for h in headvals:
        budgetHeads.append(h.text.replace(" ", ""))
    for count, d in enumerate(vals):
        if count != 6 and (-len(budgetHeads) <= count < 0):
            budgetdata[budgetHeads[count]]=d.text
    budgetVals.append(budgetdata)
    summary['Budget']=budgetVals
    
    
# #     processing beneficiary part
    beneficiaryHeads = ['No', 'Village', 'BeneficiaryTotal', 'BeneficiaryWomen', 'Households']
    raw_head = []
    beneficiaryVals = []
    beneficiarydata = {}
    benehost = {}
    bene_soup = BeautifulSoup(val, 'html.parser')
    hostbenedata = bene_soup.find('div', attrs={'id': 'p_beneficary'}).find('div').find('table')
    beneheadvals = hostbenedata.findAll('th')
    benevals = hostbenedata.findAll('td')
    for h in beneheadvals:
        raw_head.append(h.text.replace(" ", ""))
    for bene_count, bene_d in enumerate(benevals):
        if bene_count < 5:
            beneficiarydata[beneficiaryHeads[bene_count]]=cleanSpecialCharAndSpace(bene_d.text)
    beneficiaryVals.append(beneficiarydata)
    summary['Beneficiary']=beneficiaryVals


# # processing output part
    outputHeads = []
    outputVals = []
    outputdata = {}
    outputhost = {}
    output_soup = BeautifulSoup(val, 'html.parser')
    outputhostdata = output_soup.find('div', attrs={'id': 'p_outputs'}).find('div').find('table')
    outputheadvals = outputhostdata.findAll('th')
    outputvals = outputhostdata.findAll('td')
    for h in outputheadvals:
        outputHeads.append(h.text.replace(" ", ""))
    for out_count, out_val in enumerate(outputvals):
        if out_count < 6:
            outputdata[outputHeads[out_count]]=cleanSpecialChar(out_val.text)
    outputVals.append(outputdata)
    summary['Outputs_Estimated_Cost']=outputVals


# # processing technical part
    tech_soup = BeautifulSoup(val, 'html.parser')
    tech_main_data = tech_soup.find('div', attrs={'id': 'p_clearance'})
    techform = {}
    
#   Evironmental safeguard clearance 
    envHeads = []
    envVals = []
    envdata = {}
    envhost = {}
    tech_env_data = tech_main_data.find('div', attrs={'id' : 'divEA'})
    envheadvals = tech_env_data.findAll('th')
    envvals = tech_env_data.findAll('td')
    for e in envheadvals:
        envHeads.append(e.text.replace(" ", ""))
    for e_count, e_val in enumerate(envvals):
        if e_count in envHeads:
            envdata[envHeads[e_count]]=cleanSpecialChar(e_val.text)
    envVals.append(envdata)
    techform['Evironmental_Safeguard']=envVals
    
#   Land safeguard clearance
    landHeads = []
    landVals = []
    landdata = {}
    landhost = {}
    tech_land_data = tech_main_data.find('div', attrs={'id' : 'divLAR'})
    landheadvals = tech_land_data.findAll('th')
    landvals = tech_land_data.findAll('td')
    for l in landheadvals:
        landHeads.append(l.text.replace(" ", ""))
    for l_count, l_val in enumerate(landvals):
        landdata[landHeads[l_count]]=cleanSpecialChar(l_val.text)
    landVals.append(landdata)
    techform['Land_Safeguard']=landVals
    
#   Highland people safeguard clearance   
    highHeads = []
    highVals = []
    highdata = {}
    highhost = {}
    tech_highland_data = tech_main_data.find('div', attrs={'id' : 'divHP'})
    highheadvals = tech_highland_data.findAll('th')
    highvals = tech_highland_data.findAll('td')
    for h in highheadvals:
        highHeads.append(h.text.replace(" ", ""))
    for h_count, h_val in enumerate(highvals):
        highdata[highHeads[h_count]]=cleanSpecialChar(h_val.text)
    highVals.append(highdata)
    techform['Highland_Safeguard']=highVals
    
#   Sign-Off
    signHeads = []
    signVals = []
    signdata = {}
    signhost = {}
    tech_signoff_data = tech_main_data.find('div', attrs={'id' : 'pSignOff'})
    signheadvals = tech_signoff_data.findAll('th')
    signvals = tech_signoff_data.findAll('td')
    for s in signheadvals:
        signHeads.append(s.text.replace(" ", ""))
    for s_count, s_val in enumerate(signvals):
        signdata[signHeads[s_count]]=cleanSpecialChar(s_val.text)
    signVals.append(signdata)
    techform['Signoff']=signVals
    summary['Technical_Clearance']=techform
    
    
    return summary

In [10]:
### Converting contract_html to json objects
def getCHTML(val, idval):
    contract_summary = {}
    
# #     inserting id value to contract summary
    contract_summary['Id']=idval
    
# #     processing contract summary information
    csumHeads = []
    csumVals = []
    csumdata = {}
    csum_soup = BeautifulSoup(val, 'html.parser')
    csumhostdata = csum_soup.find('div', attrs={'id': 'dhead'}).find_all('table')
    try: 
        csumheadvals = csumhostdata[0].findAll('th')
        csumvals = csumhostdata[0].findAll('td')
        csumheadonevals = csumhostdata[1].findAll('th')
        csumonevals = csumhostdata[1].findAll('td')
        for cs in csumheadvals:
            csumHeads.append(cs.text.replace(" ", ""))
        for cso in csumheadonevals:
            csumHeads.append(cso.text.replace(" ", ""))
        for csov in csumonevals:
            csumvals.append(csov)
        for count, csval in enumerate(csumvals):
            csumdata[csumHeads[count]]=cleanSpecialChar(csval.text) 
    except: 
        print('failed to get contract summary')
             
    csumVals.append(csumdata)
    contract_summary['ContractSummary']=csumVals


    
# #     processing budget part
    budgetHeads = []
    budgetVals = []
    rawVals = []
    rawbvals = []
    budgetdata = {}
    soup = BeautifulSoup(val, 'html.parser')
    hostdata = soup.find('div', attrs={'id': 'dbudget'}).find('table')
    budgetheadvals = hostdata.findAll('th')
    budgetvals = hostdata.findAll('td')
    for h in budgetheadvals:
        budgetHeads.append(h.text.replace(" ", ""))
    for budget_count, budget_val in enumerate(budgetvals):
        rawbvals.append(cleanSpecialChar(budget_val.text))
    for bvc in range(int(len(rawbvals)/2)):
        bvcval = bvc+1
        btdval = soup.find('div', attrs={'id': 'dbudget'}).find('table').find('tr', attrs={'id': 'row_cb_'+str(bvcval)+''}).findAll('td')
        bvform = {}
        for c, td in enumerate(btdval):
            bvform[budgetHeads[c]]=td.text
        budgetVals.append(bvform)
    contract_summary['Budget']=budgetVals
    
    
# #     processing bidding part
    biddingHeads = ["No"]
    raw_head = []
    bidVals = []
    biddata = {}
    bidhost = {}
    rawdatalist = []
    bid_soup = BeautifulSoup(val, 'html.parser')
    hostbiddata = bid_soup.find('div', attrs={'id': 'dlbid'})
    if hostbiddata is not None:    
        hostbiddatatable = hostbiddata.find('table')
        bidheadvals = hostbiddata.findAll('th')
        bidvals = hostbiddata.findAll('td')
        bidtrs = hostbiddata.find('tbody').findAll('tr')
        for h in bidheadvals:
            biddingHeads.append(h.text.replace(" ", ""))
        del biddingHeads[1]
        del biddingHeads[-1]
        for b_count, b_tr in enumerate(bidtrs):
            numval = b_count+1
            bidtdval = hostbiddata.find('tbody').find('tr', attrs={'id': 'tbbs_'+str(numval)+''}).findAll('td')
            bidform = {}
            for c, td in enumerate(bidtdval):
                bidform[biddingHeads[c]]=cleanSpecialCharAndSpace(td.text)
            bidVals.append(bidform)
        contract_summary['Bidding']=bidVals



# # processing output part
    outputHeads = []
    outputVals = []
    outputdata = {}
    outputhost = {}
    output_soup = BeautifulSoup(val, 'html.parser')
    outputhostdata = output_soup.find('div', attrs={'id': 'c_outputs'}).find('div').find('table')
    outputheadvals = outputhostdata.findAll('th')
    outputvals = outputhostdata.findAll('td')
    for h in outputheadvals:
        outputHeads.append(h.text.replace(" ", ""))
    for out_count, out_val in enumerate(outputvals):
        if out_count < 9:
            outputdata[outputHeads[out_count]]=cleanSpecialChar(out_val.text)
        else:
            outputdata[outputHeads[-2]]=outputHeads[-1]
        outputdata[outputHeads[-2]]=outputHeads[-1]
    outputVals.append(outputdata)
    #print("\n")
    contract_summary['Outputs_Actual_Cost']=outputVals
    

    
    
# # processing progress part
    pureprogressVals = []
# # processing progress/work part
    workHeads = []
    workVals = []
    rawwvals = []
    workdata = {}
    workhost = {}
    work_soup = BeautifulSoup(val, 'html.parser')
    workhostdata = work_soup.find('div', attrs={'id': 'd_actual_work'}).find('table')
    workheadvals = workhostdata.findAll('th')
    workvals = workhostdata.findAll('td')
    for h in workheadvals:
        workHeads.append(h.text.replace(" ", ""))
    for work_count, work_val in enumerate(workvals):
        rawwvals.append(cleanSpecialChar(work_val.text))
    workdata[workHeads[0]]=cleanSpecialChar(rawwvals[0])
    workdata[workHeads[1]]=cleanSpecialChar(rawwvals[1])
    try:
        workdata['ActualworkstartdateDescription']=cleanSpecialChar(rawwvals[2])
        workdata['ActualworkcompletiondateDescription']=cleanSpecialChar(rawwvals[3])
    except:
        print("")
    pureprogressVals.append(workdata)


# # processing progress/progress part
    progressHeads = []
    progressVals = []
    rawpvals = []
    progressdata = {}
    progresshost = {}
    progress_soup = BeautifulSoup(val, 'html.parser')
    progresshostdata = progress_soup.find('div', attrs={'id': 'dlprogress'}).find('table')
    progressheadvals = progresshostdata.findAll('th')
    progressvals = progresshostdata.findAll('td')
    for h in progressheadvals:
        progressHeads.append(h.text.replace(" ", ""))
    for progress_count, progress_val in enumerate(progressvals):
        rawpvals.append(cleanSpecialChar(progress_val.text))
    for pvc in range(int(len(rawpvals)/4)):
        pvcval = pvc+1
        ptdval = progress_soup.find('div', attrs={'id': 'dlprogress'}).find('table').find('tr', attrs={'id': 'row_cp_'+str(pvcval)+''}).findAll('td')
        pvform = {}
        for c, td in enumerate(ptdval):
            pvform[progressHeads[c]]=td.text
        progressVals.append(pvform)
    pureprogressVals.append(progressVals)
    contract_summary['progress']=pureprogressVals
    
        
# #     processing payment part
    paymentHeads = []
    raw_head = []
    paymentVals = []
    paymentdata = {}
    payhost = {}
    pform = {}
    pay_soup = BeautifulSoup(val, 'html.parser')
    hostpaydata = pay_soup.find('div', attrs={'id': 'c_payment'}).find('div').find('table')
    payheadvals = hostpaydata.findAll('th')
    payvals = hostpaydata.find('tr').findAll('tr')
    for h in payheadvals:
        paymentHeads.append(h.text.replace(" ", ""))
    payvals.pop()
    payvals.pop()
    payvals.pop()
    for pay_count, pay_d in enumerate(payvals):
        rawpvalso = []
        rawpvals = []
        rawpformat = {}
        rawpvals.append(rawpformat)
        rawpvalso.append(pay_d.text.replace(" ", ""))
        respval = rawpvalso[0].replace(",", "").replace("\n", ",")
        resphost = respval.split(",")
        del resphost[0]
        resphost.pop()
        for p_count, p_v in enumerate(resphost):
            rawpformat[paymentHeads[p_count]]=cleanSpecialCharAndSpace(p_v)
        paymentVals.append(rawpformat)            
    pform[paymentHeads[-11]+"Gross"]=cleanSpecialCharAndSpace(paymentHeads[-10])
    pform['ActualPaymentCumulativeToDate(A)Net']=cleanSpecialCharAndSpace(paymentHeads[-9])
    pform['ActualPaymentCumulativeToDate(A)Tax']=cleanSpecialCharAndSpace(paymentHeads[-8])
    pform[paymentHeads[-6]+"Gross"]=cleanSpecialCharAndSpace(paymentHeads[-5])
    hostpaytext = pay_soup.find('div', attrs={'id': 'c_payment'}).findAll('p')
    if hostpaytext is not None:
        pform['TaxDescription']= hostpaytext[1].text
    paymentVals.append(pform)
    contract_summary['payment']=paymentVals
    
    
    return contract_summary


In [11]:
for i in range(len(linkinghost)):
    print(i)
    for j in host[i]['projects']:
        #pidval = splitPID(j['project_link'])
        #idval = splitPID(j['contract_link'])
        pidval = ''
        if 'project_link' in j and 'project_html' in j:
            pidval = splitPID(j['project_link'])
            single_p_summary=getPHTML(j['project_html'], pidval)
            hostList.append(single_p_summary)
        
        idval = ''
        if 'contract_link' in j and 'contract_html' in j:
            idval = splitPID(j['contract_link'])
            single_c_summary=getCHTML(j['contract_html'], idval)
            chostList.append(single_c_summary)
        
    #location_string = host[i]["province"] + " --- " + host[i]["district"] +  " --- " + host[i]["commune"]
    write_data(hostList, "data/project_detail_list.json")
    write_data(chostList, "data/contract_detail_list.json")


0
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
1
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
2
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
3
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
4
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
5
failed to get contract summary
failed to get contract summ

failed to get contract summary
36
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
37
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
38
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
39
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
40
failed to get contract s

failed to get contract summary
failed to get contract summary
71
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
72
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
73
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
74
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summ

failed to get contract summary
failed to get contract summary
failed to get contract summary
110
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
111
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
112
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
113
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
114
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
failed to get contract summary
115
failed to get c

IndexError: list index out of range

In [ ]:
def pidMatcher(val):
    for i in hostList:
        if len(i) != 0 and i['Pid'] is not None:
            if i['Pid'] == val:
                return i
            else:
                return None

In [ ]:
def idMatcher(val):
    for i in chostList:
        if len(i) != 0 and i['Id'] is not None:  
            if i['Id'] == val:
                return i
            else:
                return None

In [ ]:
#for i in range(len(linkinghost)):
for i in range(2):
    lhost = {}
    pjlist = []
    lhost['location']=linkinghost[i]['location']
    print("\nLinkinHost\n" + str(i))
    for index,j in enumerate(linkinghost[i]['projects']):
        pidval = ''
        if 'project_link' in j:
            pidval = splitPID(j['project_link'])
        idval = ''
        if 'contract_link' in j:
            idval = splitPID(j['contract_link'])
        resp = pidMatcher(pidval)
        resc = idMatcher(idval)
        if resp is not None:
            j['project_data']=resp
        if resc is not None:
            j['contract_data']=resc
        pjlist.append(j)
        #print("\n")
        #print(index)
        #if ('project_name' in j):
            #print(j['project_name'])
        #print("\n")
    lhost['projects']=pjlist
    write_data(lhost, "final_results.json")

In [ ]:
test = [1,2]

test[8] = 0
